# Information extraction from Madison city crime incident reports using Deep Learning

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
<ul class="toc-item">
<li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li>
<li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-2">Prerequisites</a></span></li>
<li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li>
<li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-4">Data preparation</a></span></li>
<li><span><a href="#EntityRecognizer-model" data-toc-modified-id="EntityRecognizer-model-5">EntityRecognizer model</a></span></li>
<ul class="toc-item">
<li><span><a href="#Finding-optimum-learning-rate" data-toc-modified-id="Finding-optimum-learning-rate-5.1">Finding optimum learning rate</a></span>    
<li><span><a href="#Model-training" data-toc-modified-id="Model-training-5.2">Model training</a></span>
<li><span><a href="#Evaluate-model-performance" data-toc-modified-id="Evaluate-model-performance-5.3">Evaluate model performance</a></span>
<li><span><a href="#Validate-results" data-toc-modified-id="Validate-results-5.4">Validate results</a></span></li>
<li><span><a href="#Save-and-load-trained-models" data-toc-modified-id="Save-and-load-trained-models-5.5">Save and load trained models</a></span></li>
</ul>
<li><span><a href="#Model-inference" data-toc-modified-id="Model-inference-6">Model inference</a></span></li>
<li><span><a href="#Publishing-the-results-as-feature-layer" data-toc-modified-id="Publishing-the-results-as-feature-layer-7">Publishing the results as feature layer</a></span></li>
<li><span><a href="#Visualize-crime-incident-on-map" data-toc-modified-id="Visualize-crime-incident-on-map- 8">Visualize crime incident on map</a></span></li>
<li><span><a href="#Create-a-hot-spot-map-of-crime-densities" data-toc-modified-id="Create-a-hot-spot-map-of-crime-densities-9">Create a hot spot map of crime densities</a></span></li>
<li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-10">Conclusion</a></span></li>
<li><span><a href="#References" data-toc-modified-id="References-11">References</a></span></li>
</ul></div>

# Introduction

This notebook investigate creating a new named entity called `“Crime”` and our ability to generate police reports that are not readily available. 

A police report writing handbook will be used to prompt a large language model (LLM) to generate police reports where the “Crime” entity will be used in proper domain context. 

Generated reports will then be used to retrain a NER model to identify the “Crime” entity within legal documents. This will be valuable in sorting, categorizing, and exploring future investigation legal documents used in court cases.  


### Reading the Report Writing Manual
Link: https://www.csus.edu/campus-safety/police-department/_internal/_documents/rwm.pdf

# Necessary Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Spacy_ner

/content/drive/MyDrive/Colab Notebooks/Spacy_ner


In [3]:
%ls -ltr

total 288
-rw------- 1 root root 149564 Mar 28  2020 Corona2.json
-rw------- 1 root root   1678 May 21 12:12 base_config.cfg
-rw------- 1 root root  34312 Jun  6 22:46 training_data.spacy
-rw------- 1 root root   2817 Jun  6 22:46 config.cfg
drwx------ 2 root root   4096 Jun 11 02:19 model-last/
drwx------ 2 root root   4096 Jun 11 02:19 model-best/
-rw------- 1 root root  96389 Jun 11 02:38 prompt_engineering.ipynb


In [4]:
import pandas as pd
import zipfile,unicodedata
# from itertools import repeat
from pathlib import Path
import re
import os
import datetime

# Data Rule Exploration 

When we started, we found success when we started with the verb that described the crime. Ensure that the format is correct and then build context around that word. With a sentence on each side. 

It was overwhelming to create one report that fit all the requirements. The goal is to provide context that would be found in a police report. Even if it was a small subset. That gets us into our NER faster. 

The words are not perfect however get us started by providing context.

In [5]:
verbs_by_crime = {
    "Violent_Crimes": [
        'Murdered', 'Assaulted', 'Raped', 'Robbed', 'Kidnapped',
        'Beat', 'Stabbed', 'Shot', 'Strangled', 'Abused',
        'Threatened', 'Attacked', 'Mugged', 'Harmed', 'Injured'
        'Harassed', 'Stalked','Menaced', 'Abducted', 'Kidnapped',
        'Slashed', 'Choked'
    ],
    "Property_Crimes": [
        'Mugged', 'Robbed', 'Burglarized', 'Stolen', 'Vandalized', 
        'Trespassed', 'Arsoned','Damaged', 'Graffitied', 'Embezzled',
        'Shoplifted', 'Sabotaged','Looted', 'Counterfeited', 'Pocketed',
        'Stripped', 'Intruded'
    ],
    "Financial_Crimes": [
        'Defrauded', 'Laundered', 'Scammed', 'Swindled', 'Falsified',
        'Concealed', 'Misappropriated', 'Extorted', 'Manipulated', 'Bribed',
        'Racketeered', 'Evaded', 'Cheated', 'Forged', 'Misused'
    ],
    "Cybercrimes": [
        'Hacked', 'Phished', 'Stole data', 'Cyberbullied', 'Spoofed',
        'Breached', 'Doxxed', 'Distributed malware', 'Hijacked accounts', 'Ransomware attack',
        'Cyberstalking', 'Distributed denial-of-service (DDoS) attack', 'Identity theft',
        'Cyber fraud', 'Password cracking'
    ],
    "Drug_Crimes": [
        'Possessed', 'Trafficked', 'Manufactured', 'Smuggled', 'Sold',
        'Cultivated', 'Distributed', 'Synthesized', 'Abused', 'Peddled',
        'Dealt', 'Transported', 'Prescribed illegally', 'Falsified prescriptions', 'Produced illicit substances'
    ],
    "White-Collar_Crimes": [
        'Committed fraud', 'Engaged in insider trading', 'Embezzled funds',
        'Bribed officials', 'Manipulated financial records', 'Conducted money laundering',
        'Engaged in corporate espionage', 'Defrauded investors', 'Orchestrated Ponzi schemes',
        'Engaged in tax evasion', 'Falsified documents', 'Misused funds',
        'Misrepresented financial statements', 'Engaged in kickbacks', 'Violated antitrust laws'
    ],
    "Sexual_Crimes": [
        'Raped', 'Assaulted sexually', 'Harassed', 'Molested', 'Exploited',
        'Fondled', 'Groped', 'Exhibited indecent exposure', 'Engaged in non-consensual acts', 'Coerced',
        'Victimized', 'Voyeurism', 'Produced child pornography', 'Solicited prostitution', 'Sextortion'
    ],
    "Public_Order_Crimes": [
        'Intoxicated publicly', 'Disturbed the peace', 'Engaged in disorderly conduct',
        'Loitered', 'Urinated in public', 'Engaged in public drunkenness', 'Panhandled', 'Engaged in public nudity',
        'Engaged in public lewdness', 'Engaged in street racing', 'Created public disturbances', 'Organized illegal gatherings',
        'Begged', 'Engaged in public gambling', 'Solicited in public'
    ],
    "Traffic_and_Motor_Vehicle_Crimes": [
        'Speeded', 'Drove under the influence (DUI)', 'Reckless driving', 'Hit and run', 'Drove without a license',
        'Texted while driving', 'Street racing', 'Violated traffic laws', 'Failed to yield', 'Ran red lights',
        'Distracted driving', 'Improper lane change', 'Driving with expired registration', 'Driving with suspended license',
        'Carjacked'
    ]
}


In [6]:
def simple_prompt(crime):
    return f"I give you '{crime}' can you put into a 3 sentences that would be found in a police report in a chorological order"

In [7]:
def prepare_crime_prompts(verbs_by_crime):
    total_prompts = {}
    for category_key, values in verbs_by_crime.items():
        print(category_key)
        prompts = [simple_prompt(x) for x in values]
        total_prompts.update({category_key:prompts})
    return total_prompts


prepare_crime_prompts(verbs_by_crime)

Violent_Crimes
Property_Crimes
Financial_Crimes
Cybercrimes
Drug_Crimes
White-Collar_Crimes
Sexual_Crimes
Public_Order_Crimes
Traffic_and_Motor_Vehicle_Crimes


{'Violent_Crimes': ["I give you 'Murdered' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Assaulted' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Raped' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Robbed' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Kidnapped' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Beat' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Stabbed' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Shot' can you put into a 3 sentences that would be found in a police report in a chorological order",
  "I give you 'Strangled' 

In [8]:
#TODO interact with open AI, Manually at the beginning

# I give you "Assaulted" can you put into a 3 sentences that would be found in a police report in a chorological order?

# I give you X can you put into a 3 sentences that would be found in a police report in a chorological order?
# We will start with 3 sentences that would pick up the context. Larger reports can be exsperimented with in the future.


In [9]:
!pip install spacy[transformers]
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.6 MB/s eta 0:00:00
2023-06-11 02:39:07.807909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 02:39:08.842705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 02:39:10.704817: I tensorflow/compiler/xla/stream_execu

In [10]:
import spacy
 
nlp = spacy.load("en_core_web_sm")
print(nlp.pipe_names)

text = "What video sharing service did Steve Chen, Chad Hurley, and Jawed Karim create in 2005?"
doc = nlp(text)
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [15]:
!pip install spacy[transformers]

NotImplementedError: ignored

In [11]:
# Tutorial on retraining SpaCy model
#  https://newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3
import json
 
with open('Corona2.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

In [12]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}
for example in data['examples']:
    temp_dict = {}
    temp_dict['text'] = example['content']
    temp_dict['entities'] = []
    for annotation in example['annotations']:
        start = annotation['start']
        end = annotation['end']
        label = annotation['tag_name'].upper()
        temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)

In [13]:
training_data['annotations'][0]

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
 'entities': [(360, 371, 'MEDICINE'),
  (383, 408, 'MEDICINE'),
  (104, 112, 'MEDICALCONDITION'),


In [14]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [15]:

from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 31/31 [00:00<00:00, 276.59it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [ ]:
# Create a config
# https://spacy.io/usage/training#quickstart

In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!python -m spacy init fill-config base_config.cfg config.cfg

2023-06-11 02:40:50.668825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
import time
# This took a really long time and needs to be solved

start = time.time()
# !python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0
end = time.time()

2023-06-11 02:41:51.065992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-11 02:42:00,829] [INFO] Set up nlp object from config
[2023-06-11 02:42:00,844] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-11 02:42:00,847] [INFO] Created vocabulary
[2023-06-11 02:42:00,849] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if yo

In [20]:
print('How long it takes to train a spaCy NER model')
print(end - start)


How long it takes to train a spaCy NER model
2021.9141142368317


In [21]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [22]:
## These are notes around police report rules to expand upon the three sentences
intro = '''

You are showing me how to fill out a police report. I would like you to generate random information for each section. 
Example
A police reports have sections of a Cover Sheet

'''

In [23]:
cover_sheet = '''

Cover Sheet Information:
Offense: Enter the applicable numerical code section and source for the crime being reported.   If multiple crimes are being charged, this field shall contain the most serious offense. 
Report Number:  Enter the a report number, preceded by the two digit year. 
In Custody Checkbox:  Check this box if the case involves an in‐custody arrest.   
Cite & Release Checkbox:  Check this box if the case involves a cite and release. 
Warrant Request Checkbox.  Check this box if the case is a warrant request. 
Date/Time of Offense: Generate a day and time at least 10 days before today’s date.  
Date/Time of Arrest:  Generate a day and time after.  
Victim #1.  Enter the last name, first name, and middle name of the primary victim. 
Victim #2.   Enter the last name, first name, and middle name of the secondary victim.  If there is no secondary victim, leave blank. 
Suspect: Enter the last name, first name, and middle name of the suspect. 
Age:  Enter the age of the suspect
Charge: Enter all charges and source for the crime or crimes being reported.  Enter one charge per line.
Submitting Officer.  Enter the first initial, last name, and badge number of the submitting officer. 
Detail/Patrol:  Enter Patrol number 
Phone: Enter the ten-digit department telephone number
Reviewing Officer:  Enter the first initial, last name, and badge number of the reviewing officer. Date/Time Submitted:  Enter the date and time the report was reviewed by the reviewing officer.

'''

In [24]:
all_reports = '''
An effective police report is always:
1. Factual.  A police report is an objective accounting of the relevant and observed facts of the
case, and any conclusions made by the reporting officer must be supported by articulated
and documented facts. Unsubstantiated opinions or conclusions are never to be included
in an effective report.

2. Accurate: Accuracy is achieved by carefully, precisely, and honestly reporting of all relevant information.

3. Clear: A police report speaks for the reporting officer when he or she is not present. There
should be no doubt or confusion regarding what happened during an incident or crime,
based upon the content of a police report.  Clarity in report writing is achieved by clear and
logical organization of information, the judicious use of simple, common, and first person
language, and effective writing mechanics.

4. Concise.  Reports should be brief but also contain all relevant information necessary for a
complete understanding of the crime or incident, without additional explanation.   Brevity
should never take precedence over accuracy, completeness, or clarity in report writing.

5. Complete. A complete report will contain all the relevant facts, information, and details that
the reader will need to have in order to have a comprehensive understanding of the crime
or incident described in the report.  The report is complete when it is a complete word
picture of the incident, there are no questions left unanswered by the reader, officer actions
are explained and justified by the contents of the report, and both supporting and
conflicting information is included.

6. Timely. No decisions can be made or actions taken regarding an arrest or request for follow
up investigation if a report is not submitted in a timely fashion.  

'''

In [25]:
all_reports

'\nAn effective police report is always:\n1. Factual.  A police report is an objective accounting of the relevant and observed facts of the\ncase, and any conclusions made by the reporting officer must be supported by articulated\nand documented facts. Unsubstantiated opinions or conclusions are never to be included\nin an effective report.\n\n2. Accurate: Accuracy is achieved by carefully, precisely, and honestly reporting of all relevant information.\n\n3. Clear: A police report speaks for the reporting officer when he or she is not present. There\nshould be no doubt or confusion regarding what happened during an incident or crime,\nbased upon the content of a police report.  Clarity in report writing is achieved by clear and\nlogical organization of information, the judicious use of simple, common, and first person\nlanguage, and effective writing mechanics.\n\n4. Concise.  Reports should be brief but also contain all relevant information necessary for a\ncomplete understanding of t

In [26]:
# Examples of Poorly Organized vs Well Organized 
example_poor_vs_well_organized = """
Poorly Organized Example 1: 
‘’’
When we arrived, the husband let us into the house. We were responding to a 9‐1‐1 call. My partner and I had been dispatched to an incident of domestic violence. A woman called for help to keep her husband from beating her. 
“”
Well Organized Example 1: 
‘’’
My partner and I were dispatched to a domestic violence incident after a woman dialed 9‐1‐1. The woman called for help because she was afraid her husband would beat her. When we arrived, the husband let us into the house.
‘’’
Poorly Organized Example 2: 
‘’’
Marie Parker said her husband refused to answer the door at first when he heard the man on the other side begin to shout. I took her statement approximately 45 minutes after the assault took place. She was sitting in the family room when her husband went to see who was at the door.
“”
Well Organized Example 2: 
‘’’
I took Marie Parker’s statement approximately 45 minutes after the assault took place. Parker said she was sitting in the family room when her husband went to see who was at the door. Initially her husband refused to answer the door when he heard the man on the other side begin to shout.  
‘’’
"""

In [ ]:
# example_poor_vs_well_organized

In [27]:
example_abstract_vs_contrete_lang = """Reports should be written using simple, common, and concrete language whenever possible.   
The use of simple language can help keep reports concise and brief, and addresses relevant information quickly and clearly. 
The following table presents examples of abstract words and phrases, along with more concrete alternatives. 
Here are examples of the abstract words and the concrete version:

Abstract Words Example 1: A number of …
Concrete Words Example 1: Seven …
Abstract Words Example 2: At a high rate of speed…
Concrete Words Example 2: 75 MPH…
Abstract Words Example 3: Appeared intoxicated…
Concrete Words Example 3: Breath smelled of an alcoholic beverage…
Abstract Words Example 4: Hostile behavior…
Concrete Words Example 4: Repeatedly struck at officers…
Abstract Words Example 5: Physical confrontation…
Concrete Words Example 5: Fight…
Abstract Words Example 6: Verbal altercation…
Concrete Words Example 6: Argument…
Abstract Words Example 7: Extensive record…
Concrete Words Example 7: Six DUI offenses over two years…
Abstract Words Example 8: Employed…
Concrete Words Example 8: Used…
Abstract Words Example 9: Dispute…
Concrete Words Example 9: Argument…
Abstract Words Example 10: Inquired…
Concrete Words Example 10: Asked…
Abstract Words Example 11: In the vicinity of…
Concrete Words Example 11: Near…
Abstract Words Example 12: Articulated…
Concrete Words Example 12: Said, told…
Abstract Words Example 13: Employed…
Concrete Words Example 13: Said, told…
Abstract Words Example 14: Employed…
Concrete Words Example 14: Said, told…
"""

In [28]:
casualty_medical_aid_report_rules = '''
While casualty reports are typically nothing more than an incident report, their importance cannot be underestimated.
The potential for civil liability from incidents in which an involved party is injured can be quite high, 
depending upon the circumstances.As a result, the need to carefully document the incident is of an utmost necessity. T
he following are elements that need to be addressed in a medical aid or casualty report. 

1. Describe the scene.  Be as thorough as possible, and include any broken concrete, improper lighting, 
incorrect signage, or other conditions observed. 

2. Establish the timeframe of the incident. This information is critical to impeach and rehabilitate the statements of 
involved parties.
3. Take a complete statement from all parties involved.Include statements detailing the victim’s injuries, and be sure to 
speak with the victim. Be as complete and thorough as possible, and if something doesn’t make sense, get clarification 
immediately, because it may be the only time the party is contacted.

4. Get complete contact information for all parties.Be sure to get alternate telephone numbers and email addresses, 
whenever possible.

5. Canvass the area for possible witnesses.Don’t hesitate to knock on doors, if necessary.

6. Describe any injuries or other preexisting medical conditions described by involved parties.
A thorough description contemporaneous to the incident will prevent possible statement changes later.

7. Take photographs of the scene, and of all involved parties. Once again, a picture is worth a thousand words. 

8. Determine if there is video of the incident. If there is video, obtain a copy, and book it as evidence. 

9. Get medical release statements, if necessary.Having access to medical records from the outset can sometimes 
prevent excessive claims at a later time. 

10. Document the fire and medical units on scene. If the involved party refuses medical aid, document the reason.   

11. Obtain the hospital information, if the involved party is transported.  Be sure to include this information in the report.
'''

theft_burglary_property_crime_report_rules = '''
Theft, burglary and other property crime reports should answer questions regarding modus operandi, 
points of entry, items taken, timeframe, and evidentiary information in order to enable investigators 
to link specific incidents together.  

The following are elements that should be addressed by an effective property crime report. 

1. Describe the scene.  Always describe the scene as it was when the victim discovered the crime, 
and also how the scene appeared when you arrived.   

2. Establish what crime occurred. Articulate all elements of the crime in the report. 

3. Establish the timeframe of the crime. This information is critical to impeach and rehabilitate the statements of 
suspects and victims. 

4. Take a complete statement from all parties involved.  Be as complete and thorough as possible, and if something 
doesn’t make sense, get clarification immediately, because it may be the only time the party is contacted.    

5. Get complete contact information for all parties.  Be sure to get alternate telephone numbers and email addresses, 
whenever possible.  Don’t list a stolen, lost or missing telephone as the only contact information in the case. 

6. Thoroughly describe the property taken, damaged, or missing.  Be as thorough as possible, and 
follow up with the victim or responsible if necessary to obtain the information.  
Be sure to include the color, make, model, value, and serial number of items, where available.    
Also describe any owner applied markings, if applicable. If the item is a cellular telephone, obtain the MEID/IMEI numbers, 
if possible.   

7. Canvass the area for witnesses.  A witness can provide suspect information, or help confirm the timeframe. 

8. Look for cameras, and obtain any video surveillance. Determine if there is any video surveillance in the area, 
and document it in the report.  Obtain copies, if possible, of the video surveillance for the timeframe of the crime, 
and book as evidence. If the surveillance is only of entrances and exits, obtain it anyway, 

9. Describe the point of entry, point of exit, and mode of theft, if possible.  Criminals are creatures of habit, 
and will typically use the same methods to commit certain types of crimes. 10. Ask the victim if any other people 
had access or permission to take their property. This can give a starting point, and also may help narrow the timeframe 
of the crime.   

11. Photograph the scene, and ask the victim if they have any pictures of their property. A picture is worth a
thousand words, every time. 

12. Look for, obtain, and book all evidence, or perceived evidence.  Look for the ninja rocks around a vehicle 
burglary with a window smash, or look for the cut cable lock in the bushes. Don’t forget to try to lift latent 
fingerprints, regardless of the value of the stolen property. 
All it takes is one print to make a case. 

13. Talk to the victim about future crime prevention techniques, if necessary. Mention LoJack for computers, 
engraving, and registration of bicycles, not leaving property unattended…an ounce of prevention is worth a pound of cure.

'''

sexual_assalt_domestic_battery_crime_report_rules = '''
Sexual assaults, domestic violence, battery, and other crimes against persons are some of the most serious 
crimes to which officers respond. The following are elements that should be addressed by an effective report 
of a sexual assault, domestic violence, battery, or other crime against persons. 

1. Describe the scene.  Always describe the scene as it was when the victim discovered the crime, and also 
how the scene appeared when you arrived. Include distances, locations of parties, lighting conditions…anything 
that may be considered relevant to the incident. 

2. Establish the timeframe of the crime.  This information is critical to impeach and rehabilitate the statements
of suspects and victims. 

3. Take a complete statement from all parties involved.  Be as complete and thorough as possible, and if something
doesn’t make sense, get clarification immediately, because it may be the only time the party is contacted.

4. Get complete contact information for all parties.Be sure to get alternate telephone numbers and email addresses, 
whenever possible.  Don’t list a stolen, lost or missing telephone as the only contact information in the case. 

5. Establish the relationships between all parties involved.  Doing so is important, because it may establish 
specific crimes, motivations, and circumstances involved in the incident.    

6. Establish what crimes occurred.  Doing so establishes probable cause for arrest.  Always ensure all elements 
of the crime are clearly articulated. 

7. Document any injuries. Take photographs, and obtain follow up photographs, if necessary. Be sure to obtain a 
medical release waiver, wherever possible.  If medical transport is necessary, document the hospital. 

8. Collect any clothing and bedding involved, and book the items as evidence. Photograph the items before booking. 

9. Document all alcohol and drug involvement by all parties. Include the amounts, types of drugs, and frequency 
of ingestion during the incident, and determine past alcohol and drug usage history.  Also determine if any of 
the parties have used alcohol or drugs together before. Be sure to document the approximate intoxication level
of all involved parties, where possible. 

10. Canvass the area for witnesses.  Check other rooms, or other businesses nearby.

11. Determine if there is video surveillance.  If so, obtain copies and book into evidence immediately.  
If the video surveillance is only of the entrance or exit of a building, obtain a copy anyway, even if the crime 
isn’t visible on the video.   

12. Consider a pretext telephone call in all sexual assault cases. Attempt to do so prior to contacting the 
suspect in the case.   Be sure to contact Rob Gold, Supervising DDA for the DA’s office SACA unit for permission prior
to conducting the call.  Gold can be reached at 916‐956-0866 (cellular), 916-874-6543(work) or 916-451-2452(home). 

13. Offer confidentiality to the victim, and offer an advocate, if applicable.  Never forget that victims of sexual 
assault and other crimes are eligible for confidentiality, and have the right to an advocate. 

14. If the crime involves sexual assault, encourage the victim to undergo an evidentiary exam. Be sure to adequately
explain the purpose of the exam, and allow the victim to make the decision. 

15. Record interviews, whenever possible.  Recording interviews is up to the individual officer, but recording interviews
ties a victim, suspect or witness to a specific statement, and limits later redactions or retractions of statements.

16. Consider the possible defenses that can be used by the suspect. When a possible defense is noted, try to rule out 
the defense though physical evidence, or follow up questioning. 

17. Do not jump to conclusions regarding the truthfulness of the victim, suspect or witness. Doing so will bias the 
initial investigation.  Always assume that the crime happened,unless there is strong evidence that indicates otherwise.",
    
'''

## ALOT OF INFORMATION

and rules! As there should be however this is our first version so lets start small then expand. Get a report that can be reproduced in a systematic way. 

There are themes between them all:

1.) Describe the scene: Describe the scene as it was when the victim discovered the crime, and also 
how the scene appeared when you arrived. Include distances, locations of parties, lighting conditions…anything 
that may be considered relevant to the incident. 



In [ ]:
# verbs_by_crime

In [29]:
# casualty_medical_aid_reports_rules

location_input = 'Boston'
date_and_time_input = 'Saturday, May 20, 2023'

In [38]:
intro = ''' You are a police report writer education tool, you need to generate data for each of the bullet points into
concise description of the crime scence
'''
describe_the_scene = '''
It is important to always describe the scene as it was:
- when the victim discovered the crime
- how the scene appeared when you arrived. 
'''

# Provide the address and any specific details about the location, such as the type of building or area.
# Date and Time: Note the date and time when the crime scene was discovered or reported.

Narrative_Text = f'''
For example purposes and generate a believeable crime location in {location_input} and {date_and_time_input}
according to these points:
- List the full address. Example {location_input} 123 Main St, Boston on the Southwest corner 
- Please describe the location
- Provide the address and any specific details about the location
- Type of building or area for a location
- Provide when the crime occured 
- Provide when the police was call
- When police arrived to the scene

Please summarize in a narrative text with chronological order

'''

    

# Initial Observations: Describe the overall appearance of the crime scene, including any signs of forced entry, disturbance, or unusual conditions.
# Physical Evidence: Identify and document any physical evidence present at the scene, such as weapons, fingerprints, footprints, bloodstains, or any other relevant items.
# Body Position and Condition: If there is a victim, describe their position, injuries, or any visible signs of trauma.
# Surroundings: Note the immediate surroundings of the crime scene, including nearby objects, furniture, or any other relevant details.
# Points of Entry or Exit: Document any possible entry or exit points that might be related to the crime.
# Environmental Factors: Take note of lighting conditions, temperature, weather conditions, or any other environmental factors that may be relevant to the investigation.
# Witnesses: Record the names and contact information of any witnesses present at the scene.
# Law Enforcement Actions: Describe any actions taken by law enforcement officers, such as evidence collection, interviews, or securing the area.



In [35]:
prompt = intro + describe_the_scene + Narrative_Text
prompt

' You are a police report writer education tool, you need to generate data for each of the bullet points into\nconcise description of the crime scence\n\nIt is important to always describe the scene as it was:\n- when the victim discovered the crime\n- how the scene appeared when you arrived.\xa0\n\nFor example purposes and generate a believeable crime location in Boston and Saturday, May 20, 2023\naccording to these points:\n- List the full address. Example Boston 123 Main St, Boston on the Southwest corner \n- Please describe the location\n- Provide the address and any specific details about the location\n- Type of building or area for a location\n- Provide when the crime occured \n- Provide when the police was call\n- When police arrived to the scene\n\nPlease summarize in a narrative text with chronological order\n\n'

In [32]:
location

NameError: ignored

In [36]:
narrative_start = f'''
I, {name}, on {date_and_time_input} received a call to {location_input} reference to (the crime). 
My investigation revealed the following information.

'''

NameError: ignored

In [34]:
date_and_time

NameError: ignored

In [ ]:
describe_the_scene + 

'\nDescribe the scene.\xa0\xa0Always describe the scene as it was when the victim discovered the crime, and also \nhow the scene appeared when you arrived.\xa0Include distances, locations of parties, lighting conditions…anything \nthat may be considered relevant to the incident. \n\n'

In [40]:
'''

1. Describe the scene.  Always describe the scene as it was when the victim discovered the crime, and also 
how the scene appeared when you arrived. Include distances, locations of parties, lighting conditions…anything 
that may be considered relevant to the incident. 


2. Establish the timeframe of the crime.  This information is critical to impeach and rehabilitate the statements
of suspects and victims. 

3. Take a complete statement from all parties involved.  Be as complete and thorough as possible, and if something
doesn’t make sense, get clarification immediately, because it may be the only time the party is contacted.

4. Get complete contact information for all parties.Be sure to get alternate telephone numbers and email addresses, 
whenever possible.  Don’t list a stolen, lost or missing telephone as the only contact information in the case. 

5. Establish the relationships between all parties involved.  Doing so is important, because it may establish 
specific crimes, motivations, and circumstances involved in the incident.    

6. Establish what crimes occurred.  Doing so establishes probable cause for arrest.  Always ensure all elements 
of the crime are clearly articulated. 

7. Document any injuries. Take photographs, and obtain follow up photographs, if necessary. Be sure to obtain a 
medical release waiver, wherever possible.  If medical transport is necessary, document the hospital. 

8. Collect any clothing and bedding involved, and book the items as evidence. Photograph the items before booking. 

9. Document all alcohol and drug involvement by all parties. Include the amounts, types of drugs, and frequency 
of ingestion during the incident, and determine past alcohol and drug usage history.  Also determine if any of 
the parties have used alcohol or drugs together before. Be sure to document the approximate intoxication level
of all involved parties, where possible. 

10. Canvass the area for witnesses.  Check other rooms, or other businesses nearby.

11. Determine if there is video surveillance.  If so, obtain copies and book into evidence immediately.  
If the video surveillance is only of the entrance or exit of a building, obtain a copy anyway, even if the crime 
isn’t visible on the video.   

12. Consider a pretext telephone call in all sexual assault cases. Attempt to do so prior to contacting the 
suspect in the case.   Be sure to contact Rob Gold, Supervising DDA for the DA’s office SACA unit for permission prior
to conducting the call.  Gold can be reached at 916‐956-0866 (cellular), 916-874-6543(work) or 916-451-2452(home). 

13. Offer confidentiality to the victim, and offer an advocate, if applicable.  Never forget that victims of sexual 
assault and other crimes are eligible for confidentiality, and have the right to an advocate. 

14. If the crime involves sexual assault, encourage the victim to undergo an evidentiary exam. Be sure to adequately
explain the purpose of the exam, and allow the victim to make the decision. 

15. Record interviews, whenever possible.  Recording interviews is up to the individual officer, but recording interviews
ties a victim, suspect or witness to a specific statement, and limits later redactions or retractions of statements.

16. Consider the possible defenses that can be used by the suspect. When a possible defense is noted, try to rule out 
the defense though physical evidence, or follow up questioning. 

17. Do not jump to conclusions regarding the truthfulness of the victim, suspect or witness. Doing so will bias the 
initial investigation.  Always assume that the crime happened,unless there is strong evidence that indicates otherwise.",

'''

'\n\n1. Describe the scene.\xa0\xa0Always describe the scene as it was when the victim discovered the crime, and also \nhow the scene appeared when you arrived.\xa0Include distances, locations of parties, lighting conditions…anything \nthat may be considered relevant to the incident. \n\n\n2. Establish the timeframe of the crime.\xa0\xa0This information is critical to impeach and rehabilitate the statements\nof suspects and victims. \n\n3. Take a complete statement from all parties involved.\xa0\xa0Be as complete and thorough as possible, and if something\ndoesn’t make sense, get clarification immediately, because it may be the only time the party is contacted.\n\n4. Get complete contact information for all parties.Be sure to get alternate telephone numbers and email addresses, \nwhenever possible.\xa0\xa0Don’t list a stolen, lost or missing telephone as the only contact information in the case. \n\n5. Establish the relationships between all parties involved.\xa0\xa0Doing so is importa

In [41]:
Report_Types.keys()

NameError: ignored

In [ ]:
# Report_Types['Theft/Burglary/Other Property Crime Reports']